In [109]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
from sodapy import Socrata
import folium
import json
import scipy
import plotly.figure_factory as ff
import datetime
import math

In [ ]:
lpd = pd.read_csv('laadpaaldata.csv')

In [ ]:
###Inladen API - kijk naar country code en maxresults
response = requests.get('https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=1000&compact=true&verbose=false&key=772ba88b-50db-49f1-b9f1-933c6053edd3')
response.status_code

In [ ]:
###Omzetten naar dictionary
json_lp = response.json()

In [ ]:
###Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
opencharge = pd.json_normalize(json_lp)
#Daarna nog handmatig kijken welke kolommen over zijn in dit geval Connections
#Kijken naar eerst laadpaal op de locatie
#Kan je uitpakken middels:
df4 = pd.json_normalize(opencharge.Connections)
df5 = pd.json_normalize(df4[0])
df5.head()
###Bestanden samenvoegen
ocm = pd.concat([opencharge, df5], axis=1)
ocm.head()

In [ ]:
# #RDW data inladen via api
# client = Socrata("opendata.rdw.nl", None)

# results = client.get("8wbe-pu7d", limit=50000)
# rdw = pd.DataFrame.from_records(results)


In [ ]:
rdw = pd.read_csv('fuel_count.csv')
rdw.rename({'datum_eerste_toelating':'date_of_acceptance','benzine_count': 'gasoline_count', 'elektriciteit_count':'electric_count', 'waterstof_count':'hydrogen_count'}, inplace=True, axis=1)

In [ ]:
ocm.info()

In [ ]:
ocm.head()

In [ ]:
lpd.info()

In [ ]:
lpd

In [ ]:
ocm.columns = ocm.columns.str.replace('.', '_') 
ocm.columns = ocm.columns.str.replace(' ', '_')

ocm = ocm.loc[:,~ocm.columns.duplicated()].copy()
ocm = ocm[ocm['Quantity'].notna()]
ocm = ocm[ocm['NumberOfPoints'] != 0]
ocm = ocm[ocm['AddressInfo_Latitude'] > 50]
ocm = ocm.drop(columns=['Connections','AddressInfo_DistanceUnit', 'GeneralComments', 'AddressInfo_RelatedURL', 'UsageCost', 'AddressInfo_AddressLine2', 'AddressInfo_AccessComments', 
                        'AddressInfo_ContactTelephone1', 'AddressInfo_ContactEmail', 'AddressInfo_ContactTelephone2', 'OperatorsReference', 'DataProvidersReference',
                        'Reference', 'Amps', 'Voltage', 'Comments'])
ocm.columns = map(str.lower, ocm.columns)
rdw.columns = map(str.lower, rdw.columns)
lpd.columns = map(str.lower, lpd.columns)

ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace('[a-zA-Z]', '')
ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace(' ', '')


In [ ]:
ocm['addressinfo_stateorprovince'].isna().sum()

In [ ]:
m = folium.Map(location=[52.091684, 5.114055], zoom_start=7)


In [ ]:
import geopandas as gpd
import geojson

In [ ]:
# r =  requests.get('https://data.opendatasoft.com/api/v2/catalog/datasets/georef-netherlands-postcode-pc4%40public/exports/geojson?limit=-1&offset=0&lang=nl&timezone=Europe%2FBerlin')

In [ ]:
# geofile = gpd.read_file('https://data.opendatasoft.com/api/v2/catalog/datasets/georef-netherlands-postcode-pc4%40public/exports/geojson?limit=-1&offset=0&lang=nl&timezone=Europe%2FBerlin')

In [ ]:
# geofile = r.json()

In [ ]:
# geo_map = {}
# for feature in geofile['features']:
#     feature['id'] = feature['properties']['prov_name']
#     geo_map[feature['properties']['prov_name']] = feature['id']

In [ ]:
# ocm['geo_id'] = ocm['addressinfo_stateorprovince'].apply(lambda x: geo_map[x])

In [ ]:
# folium.Choropleth(
#     geo_data=geofile.to_json(),
#     name="geometry",
#     data=rdw,
#     columns=["postcode", 'aantal'],
#     key_on='feature.properties.pc4_code',
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=.1,
# ).add_to(m)

In [ ]:
# fig = px.choropleth(merge, geojson=geofile, locations='addressinfo_stateorprovince', color='aantal',
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),                         
#                            scope="europe", )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.show()

In [ ]:
lpd = lpd.loc[(lpd['chargetime'] >= 0)]
lpd = lpd.loc[(lpd['connectedtime'] < 20)]

In [ ]:
fig = px.histogram(data_frame=lpd, x='chargetime', color='chargetime', color_discrete_sequence=['indianred'])
fig.update_layout({'title': {'text': 'Count of charging time'}})
fig.update_layout({'xaxis': {'title': {'text': 'Charging time (hrs)'}}})
fig.update_layout({'yaxis': {'title': {'text': 'Count'}}})
fig.update_layout(showlegend=False)
my_annotation = {'x': 2.4657452041318058, 'y': 192,'showarrow': True, 'ay': -200, 'ax':200, 'arrowhead': 3,'text': "Mean/Average",'font' : {'size': 10, 'color': 'black'}}
my_annotation2 = {'x': 2.2, 'y': 192,'showarrow': True, 'ay': -150, 'ax':-50, 'arrowhead': 3,'text': "Median",'font' : {'size': 10, 'color': 'black'}}
fig.update_layout({'annotations': [my_annotation, my_annotation2]})
hist_data = [lpd['chargetime']]
group_labels = ['Charge Time']

In [ ]:
hist_data = [lpd['chargetime'], lpd['connectedtime']]
group_labels = ['Charge Time', 'Connected Time']

In [ ]:
cht_avg = lpd['chargetime'].mean()
cht_median = lpd['chargetime'].median()
ct_avg = lpd['connectedtime'].mean()
ct_median = lpd['connectedtime'].median()

In [ ]:
df = rdw.loc[(rdw['date_of_acceptance'] == '2010-01-01')]
df

In [ ]:
rdw['total'] = rdw['gasoline_count'] + rdw['electric_count'] + rdw['diesel_count'] + rdw['hydrogen_count'] + rdw['alcohol_count'] + rdw['lpg_count'] + rdw['cng_count'] + rdw['lng_count']

In [ ]:
rdw['ratio_electric'] = rdw['electric_count'] / rdw['total'] * 100

In [ ]:
rdw['non_electric'] = rdw['total'] - rdw['electric_count']

In [ ]:
rdw['date_of_acceptance_y'] = pd.DatetimeIndex(rdw['date_of_acceptance']).year

In [ ]:
rdw = rdw.loc[(rdw['date_of_acceptance'] >= '2009-12-01')]
rdw = rdw.loc[(rdw['date_of_acceptance'] <= '2022-12-31')]

In [ ]:
rdw['date_of_acceptance'] = pd.to_datetime(rdw['date_of_acceptance'])

In [126]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X = pd.date_range(start='2010-01-01', end='2022-12-01', freq='MS')
y = rdw['electric_count']

X_train, X_test, y_train, y_test = train_test_split(rdw.date_of_acceptance_y, y)

lr = LinearRegression()
lr.fit(X_train.values.reshape(-1, 1), y_train.values)

prediction = lr.predict(X_test.values.reshape(-1, 1))

years = list(range(2023, 2051))

df = pd.DataFrame(columns=['year', 'predicted_electric_count', 'predicted_non_electric_count', 'percentage'])
pre_electric = []

for year in years:
    predicted = lr.predict(np.array([[year]]))[0]
    # print(f'predicted {year} ratio: {lr.predict(np.array([[year]]))[0]}%')
    pre_electric.append(predicted)
    
df['year'] = years
df['predicted_electric_count'] = pre_electric

In [127]:
y_non = rdw['non_electric']
X_train, X_test, y_train, y_test = train_test_split(rdw.date_of_acceptance_y, y_non)

lr = LinearRegression()
lr.fit(X_train.values.reshape(-1, 1), y_train.values)

prediction = lr.predict(X_test.values.reshape(-1, 1))

pre_non_electric = []

for year in years:
    predicted = lr.predict(np.array([[year]]))[0]
    # print(f'predicted {year} ratio: {lr.predict(np.array([[year]]))[0]}%')
    pre_non_electric.append(predicted)


df['predicted_non_electric_count'] = pre_non_electric

In [128]:
df['percentage'] = df['predicted_electric_count'] / (df['predicted_electric_count'] + df['predicted_non_electric_count']) * 100

In [129]:
df

,year,predicted_electric_count,predicted_non_electric_count,percentage
0,2023,18376.499752,41204.335146,30.842971
1,2024,19963.138673,39758.954581,33.426723
2,2025,21549.777594,38313.574016,35.998281
3,2026,23136.416515,36868.193451,38.557732
4,2027,24723.055436,35422.812886,41.105160
5,2028,26309.694357,33977.432321,43.640651
6,2029,27896.333278,32532.051756,46.164287
7,2030,29482.972199,31086.671191,48.676153
8,2031,31069.611120,29641.290626,51.176329
9,2032,32656.250041,28195.910062,53.664899


In [131]:
px.line(df, x='year', y='percentage', title='Percentage of electric cars in the Netherlands', log_y=True)